In [1]:
%load_ext autoreload
%autoreload 2
import NuLattice.smearedInteraction as smearI
import NuLattice.CCM.coupled_cluster as ccm
import NuLattice.references as ref

In [2]:
myL= 5
bpi = 0.7
c0 = -0.185
sL = 0.08
sNL = 0.08
a_lat = 1.97
tKin, inter = smearI.get_full_int(myL, bpi, c0, sL, sNL, a_lat)
holes = ref.ref_3H_gs
no2b_stuff = ccm.get_norm_ordered_ham(myL, holes, tKin, inter)

In [3]:
corrEn, t1, t2 = ccm.ccsd_solver(no2b_stuff[1], no2b_stuff[2], 
                            eps = 1e-8, maxSteps = 500, max_diis = 10, mixing = 0.5,
                            sparse=True, verbose=True)
print(f'Energy: {corrEn + no2b_stuff[0]}')

Step 0: -4332.144175208456
Step 1: -611.9852149009093 difference = 6.078837968185068
Step 2: -96.02225124942495 difference = 5.373368744617664
Step 3: -72.93159741355922 difference = 0.3166069941527538
Step 4: -87.95368022472648 difference = 0.17079538653510598
Step 5: -94.6827885700404 difference = 0.07107002705498207
Step 6: -93.56894918260156 difference = 0.011903942463489312
Step 7: -104.47067957970935 difference = 0.10435205783063718
Step 8: -99.99754582822236 difference = 0.04473243532567319
Step 9: -109.65209276728646 difference = 0.08804708323765287
Step 10: -105.52416776317116 difference = 0.039118290071518375
Step 11: -105.55719035842492 difference = 0.0003128407940910958
Step 12: -101.4006174809066 difference = 0.04099159335297917
Step 13: -109.09515267204345 difference = 0.07053049565151431
Step 14: -108.15285076251897 difference = 0.008712686747329352
Step 15: -112.69020192917606 difference = 0.040263936784040374
Step 16: -112.17208138762533 difference = 0.0046189794745833